In [76]:
import sounddevice as sd
import keras
import pandas as pd
import librosa
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer
sd.query_devices()

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 머리에 거는 수화기(QCY ArcBuds Hands-Fr, MME (1 in, 0 out)
   2 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  3 헤드폰(QCY ArcBuds Stereo), MME (0 in, 2 out)
   4 Digital Audio (S/PDIF)(High Def, MME (0 in, 2 out)
   5 머리에 거는 수화기(QCY ArcBuds Hands-Fr, MME (0 in, 1 out)
   6 EDGE 24FH3A(NVIDIA High Definit, MME (0 in, 2 out)
   7 SPDIF Out (HD Audio SPDIF out), Windows WDM-KS (0 in, 2 out)
   8 Output (NVIDIA High Definition Audio), Windows WDM-KS (0 in, 2 out)
   9 헤드폰 (), Windows WDM-KS (0 in, 2 out)
  10 머리에 거는 수화기 (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(QCY ArcBuds)), Windows WDM-KS (0 in, 1 out)
  11 머리에 거는 수화기 (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(QCY ArcBuds)), Windows WDM-KS (1 in, 0 out)

In [143]:
import queue, os, threading
import sounddevice as sd
import soundfile as sf
import time
from scipy.io.wavfile import write
import speech_recognition as sr

q = queue.Queue()
recorder = False
recording = False

def complicated_record():
    with sf.SoundFile("./micInputTest.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save):
            while recording:
                file.write(q.get())
        
def complicated_save(indata, frames, time, status):
    q.put(indata.copy())
    
def start():
    global recorder
    global recording
    recording = True
    recorder = threading.Thread(target=complicated_record)
    print('start recording')
    recorder.start()
    
def stop():
    global recorder
    global recording
    recording = False
    recorder.join()
    print('stop recording')
    
start()
time.sleep(5)
stop()

# Initialize the Recognizer
r = sr.Recognizer()
sentence = ""
# try :
audioFile = sr.AudioFile('./micInputTest.wav')
with audioFile as source :
    audio = r.record(source)
sentence = r.recognize_google(audio, language='ko-KR')
get_answer(audioFile.filename_or_fileobject,sentence)

# except :
#     print("no input")
    
print(sentence)

start recording
stop recording


C:\Users\smhrd\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
당신의 1번째 감정은 Angry일 확률 : 0.00
당신의 2번째 감정은 Disgust일 확률 : 0.00
당신의 3번째 감정은 Fear일 확률 : 0.00
당신의 4번째 감정은 Happiness일 확률 : 0.00
당신의 5번째 감정은 Neutral일 확률 : 1.00
당신의 6번째 감정은 Sadness일 확률 : 0.00
당신의 7번째 감정은 Surprise일 확률 : 0.00
난 무서워서 정말 즐거워


In [38]:
def perceptual_sharpness(audio_path, sr=16000, n_fft=512, hop_length=256):
    # 음원 파일 로드
    y= audio_path
    sr = sr

    # STFT 수행
    D = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))

    # 주파수 대역별로 에너지 계산
    energy = np.sum(D, axis=0)

    # 고주파수 대역 성분 추출
    high_freq_energy = energy[3000:8000]  # 예시로 3000Hz에서 6000Hz 사이의 주파수 대역을 고주파수 대역으로 설정

    # Perceptual Sharpness 계산
    sharpness = np.sum(np.log1p(high_freq_energy))

    return sharpness

### 음성 추출 및 벡터화 기능 만들기

In [88]:
# 임베딩 함수 설정
class text_embedding():
    def __init__(self, model_name):
        self.model_name = model_name

    def fit(self, X, y=None):
        return self
  
    def transform(self, X,raw_label):
        embedding_model = SentenceTransformer(self.model_name)
        embedding_vec = embedding_model.encode(raw_label)
        X_val = np.concatenate((X, embedding_vec))
        return X_val

In [50]:
def extract_emotion(data):
    audio, sr = librosa.load(data, sr=16000)
        ##Mel-spectrogram 구현
    spectrogram = librosa.stft(audio, n_fft=512, hop_length= 256) 
    power_spectrogram = spectrogram**2
    mel = librosa.feature.melspectrogram(S=power_spectrogram, sr=sr)
    mel = librosa.power_to_db(np.abs(mel)**2)
    #mfcc 구현
    mfccs = librosa.feature.mfcc(S = mel, n_mfcc=20)

    stft = np.abs(spectrogram)
    chroma_stft = librosa.feature.chroma_stft(S=stft,hop_length=512)
    rms = librosa.feature.rms(y=audio)
    spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
    spectral_bandwidths = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)
    zero_crossing_rates = librosa.feature.zero_crossing_rate(y=audio)
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr)
    tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)
    ps = perceptual_sharpness(audio)


    mfccs_mean = mfccs.mean(axis=1)
    mfccs_var = mfccs.mean(axis=1)

    for k in range(len(mfccs_mean)):
        locals()[f'mfccs_mean_{k}'] = mfccs_mean[k]
        locals()[f'mfccs_var_{k}'] = mfccs_var[k]
    chroma_stft_mean = chroma_stft.mean()
    chroma_stft_var = chroma_stft.var()
    rms_mean = rms.mean()
    rms_var = rms.var()
    spectral_centroids_mean = spectral_centroids.mean()
    spectral_centroids_var = spectral_centroids.var()
    spectral_bandwidths_mean = spectral_bandwidths.mean()
    spectral_bandwidths_var = spectral_bandwidths.var()
    spectral_rolloff_mean = spectral_rolloff.mean()
    spectral_rolloff_var = spectral_rolloff.var()
    zero_crossing_rates_mean = zero_crossing_rates.mean()
    zero_crossing_rates_var = zero_crossing_rates.var()
    harmony_mean = chroma_cens.mean()
    harmony_var = chroma_cens.var()
    tempo_mean = tempo.mean()
    tempo_var = tempo.var()
    perceptual_sharpness_mean = ps.mean()
    perceptual_sharpness_var = ps.var()


        #합치기
    features = np.array([])
    for j in range(20):
        features = np.hstack((features,locals()[f'mfccs_mean_{j}'],locals()[f'mfccs_var_{j}']))

    features = np.hstack((features,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroids_mean
                         ,spectral_centroids_var, spectral_bandwidths_mean, spectral_bandwidths_var,spectral_rolloff_mean
                         , spectral_rolloff_var,zero_crossing_rates_mean, zero_crossing_rates_var, harmony_mean, harmony_var
                         , tempo_mean,tempo_var,perceptual_sharpness_mean,perceptual_sharpness_var))
    return features

### 모델과 스케일러 불러와 features에 적용하기

In [129]:
def get_answer(file,sentence):
    model = keras.models.load_model('./model/model_06-0.8196.keras')
    scaler = joblib.load('./scaler.pkl')
    
    features = extract_emotion(file)
    
    
    txt_embed = text_embedding(model_name = 'jhgan/ko-sroberta-multitask')
    X_embed = txt_embed.transform(features,sentence)
    
    X_embed = np.array(X_embed).reshape(-1,826)
    
    X_scaled = scaler.transform(X_embed)
    X_scaled = np.expand_dims(X_scaled, axis = 1)
    pre = model.predict(X_scaled)

    emotion_list = ['Angry','Disgust','Fear','Happiness','Neutral','Sadness','Surprise']


    # 각 클래스에 속할 확률 출력
    for i, prob in enumerate(pre[0]):
        print(f"당신의 {i + 1}번째 감정은 {emotion_list[i]}일 확률 : {prob:.2f}")
print('입니다')

입니다
